## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-08-25-15-51-53 +0000,nypost,"Elon Musk sues Apple, Sam Altman’s OpenAI for ...",https://nypost.com/2025/08/25/business/elon-mu...
1,2025-08-25-15-48-10 +0000,nypost,"Apple plotting iPhone lineup overhaul, includi...",https://nypost.com/2025/08/25/business/apple-p...
2,2025-08-25-15-47-07 +0000,nyt,How Southwest’s New Seating Policy Will Affect...,https://www.nytimes.com/2025/08/22/travel/sout...
3,2025-08-25-15-46-11 +0000,48hills,Ficks’ Picks: Diving into CROSSROADS Fest’s ri...,https://48hills.org/2025/08/ficks-picks-diving...
4,2025-08-25-15-46-00 +0000,wsj,Tech executives including VC firm Andreessen H...,https://www.wsj.com/politics/silicon-valley-la...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2369/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
78,trump,27
23,will,11
188,ukraine,10
20,new,10
136,israeli,8


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
207,2025-08-24-15-54-42 +0000,nypost,House Dem leader Jeffries accuses Trump of try...,https://nypost.com/2025/08/24/us-news/hakeem-j...,60
11,2025-08-25-15-26-19 +0000,nypost,Trump signs executive order banning flag burni...,https://nypost.com/2025/08/25/us-news/trump-si...,59
191,2025-08-24-18-08-34 +0000,nypost,Vance reveals what Putin has conceded in Ukrai...,https://nypost.com/2025/08/24/us-news/vance-re...,58
6,2025-08-25-15-40-13 +0000,nypost,Keurig Dr Pepper to buy JDE Peet’s in $18B dea...,https://nypost.com/2025/08/25/business/keurig-...,58
76,2025-08-25-10-58-32 +0000,nypost,Trump warns ‘sloppy’ Chris Christie could be p...,https://nypost.com/2025/08/25/us-news/trump-ha...,56


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
207,60,2025-08-24-15-54-42 +0000,nypost,House Dem leader Jeffries accuses Trump of try...,https://nypost.com/2025/08/24/us-news/hakeem-j...
6,47,2025-08-25-15-40-13 +0000,nypost,Keurig Dr Pepper to buy JDE Peet’s in $18B dea...,https://nypost.com/2025/08/25/business/keurig-...
12,46,2025-08-25-15-21-00 +0000,wsj,Kilmar Abrego Garcia was arrested by immigrati...,https://www.wsj.com/us-news/law/kilmar-abrego-...
15,43,2025-08-25-15-08-19 +0000,nyt,Five Journalists Among Those Killed in Israeli...,https://www.nytimes.com/2025/08/25/world/middl...
4,35,2025-08-25-15-46-00 +0000,wsj,Tech executives including VC firm Andreessen H...,https://www.wsj.com/politics/silicon-valley-la...
192,34,2025-08-24-18-08-34 +0000,nypost,Russia’s top diplomat says Zelensky can’t sign...,https://nypost.com/2025/08/24/world-news/russi...
76,29,2025-08-25-10-58-32 +0000,nypost,Trump warns ‘sloppy’ Chris Christie could be p...,https://nypost.com/2025/08/25/us-news/trump-ha...
188,29,2025-08-24-18-30-00 +0000,nypost,Australian mom accused of killing American ste...,https://nypost.com/2025/08/24/world-news/lisa-...
129,27,2025-08-25-01-56-00 +0000,wsj,Here’s what you need to know about the unusual...,https://www.wsj.com/politics/policy/texas-cali...
54,25,2025-08-25-12-13-21 +0000,nypost,PBR die-hards line up for first taste of beer-...,https://nypost.com/2025/08/25/us-news/pbr-die-...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
